# Multilayer Perceptron

## Loading modules

In [18]:
import ipyvolume as ipv
import numpy as np
from ipywidgets import interactive, FloatSlider
import matplotlib.pyplot as plt

## Helper functions

In [10]:
def na(x):
    return np.array(x)

def draw_grid(x,y,z):
    ipv.plot_wireframe(x, y, z * np.ones(x.shape), color="black")

def show_point(x,y,z):
    x = na([float(x)])
    y = na([float(y)])
    z = na([float(z)])
    ipv.scatter(x,y,z, color="blue", marker="sphere",size=5)

## Define Neuron and MLP

In [26]:
class MultilayerPerceptron:
    def __init__(self):
        self.layers = []
        self.number_layers = 0
        self.neuronal_function = []
    
    def predict(self, **kwargs):
        neuron_input = {}
        weights = []
        bias = []
        for key in kwargs:
            if key.startswith("p"):
                neuron_input[key] = kwargs[key]
            elif key.startswith("w"):
                weights.append(key)
            elif key.startswith("b"):
                bias.append(key)
        next_layer_input = {}

        for layer_number, number_neurons in enumerate(self.layers):
            next_layer_input = {}
            for n in range(0, number_neurons):
                na = 0
                for i, p in enumerate(neuron_input):
                    # print(str(neuron_input[p]))
                    # print(str(weights[i]))
                    na += neuron_input[p] * kwargs[weights[i]]
                na += kwargs[bias[0]]

                neuron_function = self.neuronal_function[layer_number]
                if neuron_function.__class__ == list:
                    neuron_function = self.neuronal_function[layer_number][n]
                aa = neuron_function(na)
                bias = bias[1:]
                weights = weights[len(list(neuron_input.keys())):]
                next_layer_input["n" + str(n)] = aa
            neuron_input = next_layer_input

        return next_layer_input[next(iter(next_layer_input))]

    
    @staticmethod
    def generate(arch, neuronal_function):
        mlp_nn = MultilayerPerceptron()
        mlp_nn.number_layers = len(arch) - 1
        mlp_nn.input_size = arch[0]
        mlp_nn.layers = arch[1:]
        mlp_nn.neuronal_function = neuronal_function
        
        return mlp_nn

## Define Neural Transfer Functions

In [12]:
def logsig(n):
    return 1 / (1 + np.exp(n))

def purelin(n):
    return n

In [28]:
a = np.arange(-5, 5)
U, V = np.meshgrid(a, a)

target_arch = [2, 4, 4, 2, 1]
weights_def = {}
bias_def = {}

input_size = target_arch[0]
neuronal_functions = [[logsig, logsig, purelin, purelin], [logsig, logsig, purelin, purelin], [logsig, purelin], logsig]

w_index = 0
b_index = 0

for number_neuron in target_arch[1:]:
    for i in range(0, number_neuron):
        for j in range(0, input_size):
            weights_def["w" + str(w_index)] = (-1,1,0.1)
            w_index += 1

        bias_def["b" + str(b_index)] = (-1,1,0.1)
        b_index += 1
    input_size = number_neuron

nn = MultilayerPerceptron.generate(target_arch, neuronal_functions)

def f(**kwargs):
    X = U
    Y = V
    a = nn.predict(p1=X, p2=Y, **kwargs)

    ipv.figure(width=400,height=600, offline=True)    
    ipv.plot_surface(X, Y, a, color="orange")
    ipv.plot_wireframe(X, Y, a, color="red")
    show_point(0,0,0)
    draw_grid(X,Y,0)
    ipv.show()

interactive_plot = interactive(
    f, **weights_def, **bias_def
)
# for child in interactive_plot.children:
#     if child.__class__ == FloatSlider:
#         child.value = 1.0
output = interactive_plot.children[-1]
interactive_plot

interactive(children=(FloatSlider(value=0.0, description='w0', max=1.0, min=-1.0), FloatSlider(value=0.0, desc…